# Final Exam Review (Adelaide) [50 points]

*Contact TA: Emaad Ahmed Manzoor (emaad@cmu.edu)*

Recommended videos: https://www.youtube.com/playlist?list=PL5-da3qGB5ICeMbQuqbbCOQWcS6OYBr5A

## Dataset

   - We will be using the Fashion-MNIST dataset. Take a look at what the data looks like [here](https://github.com/zalandoresearch/fashion-mnist).
   - Download the data from [here](https://www.dropbox.com/s/qgsk90f22tvsqjp/fashionmnist.zip?dl=0).
   - There are 60,000 training images and 10,000 test images.
   - 10 classes: t-shirt, trouser, pullover, etc. labeled as 0 to 9.

## Q0. Load the data into variables: Xtrain, ytrain, Xtest and ytest. [5 points]

   - Each .csv files contains a header that describes each column.
   - Reading the files takes <1 minute.
   - Print the shape of each variable.

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train_df = pd.read_csv("fashion-mnist_train.csv", sep=",",  thousands=",")
test_df = pd.read_csv("fashion-mnist_test.csv", sep=",",  thousands=",")

In [3]:
Xtrain = train_df.loc[:, train_df.columns != 'label']
Xtest = test_df.loc[:, test_df.columns != 'label']
ytrain = train_df.loc[:, 'label']
ytest = test_df.loc[:,  'label']

print("Xtrain:",Xtrain.shape,"  Xtest:",Xtest.shape,"  ytest:",ytest.shape,"  ytrain:",ytrain.shape)


Xtrain: (60000, 784)   Xtest: (10000, 784)   ytest: (10000,)   ytrain: (60000,)


## Q1. Unsupervised image labeling. [10 points]

   - Cluster the images into 10 clusters via [K-Means](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).
   - Label each cluster with the most popular image class it contains.
   - Print the class label assigned to each cluster.
   - Report the overall accuracy on the test data.

Hint: You may need to convert the K-Means cluster labels to integers using the function `labels_.astype(int)`.

Clustering takes around 1 minute.

In [36]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=10, n_jobs=-1)
fit = model.fit(Xtrain)

In [37]:
for i, cluster in enumerate(fit.cluster_centers_):
    label = fit.predict(cluster)
    print('Cluster %d label %d' % (i + 1, label) )

Cluster 1 label 0
Cluster 2 label 1
Cluster 3 label 2
Cluster 4 label 3
Cluster 5 label 4
Cluster 6 label 5
Cluster 7 label 6
Cluster 8 label 7
Cluster 9 label 8
Cluster 10 label 9


In [38]:
from sklearn.metrics import accuracy_score


print(fit.predict(Xtest))
accuracy_score(ytest, fit.predict(Xtest))

[5 4 8 ..., 9 1 1]


0.055300000000000002

## Q2. Dimensionality reduction + unsupervised image labeling. [15 points]

Do this for $k = 100$ and $k = 500$.

   - Reduce the dimensionality to $k$ dimensions via [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).
   - Repeat Q1 on the reduced-dimensionality data.
   - Report the overall accuracy on the test data.
   
This takes ~1 minute.

## Q3. Supervised image labeling. [20 points]

   - Train a [logistic regression classifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression).
   - Set the classifier parameters `multi_class='multinomial'` and `solver='lbfgs'`.
   - Select the parameter `C` with the best mean accuracy score in [3-fold cross-validation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).
   - Try `C = 1.0`, `C = 100.0`.
   - Report the overall accuracy on the test data with the best `C` trained on the entire training data.

Each fold should take around 1 minute.